In [352]:
#in the absence of known webapi which can be used to retrieve the data , I am using web scrapping
#to get the corporate bond info. getting it from a webapi which will return the bond yields will be much 
#easier

In [353]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth', 200)

# URL of the source
url = 'https://www.hl.co.uk/shares/corporate-bonds-gilts/bond-prices/gbp-bonds'

# Load data from URL
tables = pd.read_html(url)
# Extract the table of interest (in this case, the first table on the page)
df = tables[0]
df=df.drop(columns=['Actions'])
df.head(10)



,Issuer,Coupon (%),Maturity,Price
0,3i Group GBP | XS0104440986 | 0924597,5.750,3 December 2032,102.290
1,A2D Funding plc GBP | XS1103286305 | BQ8NZW9,4.500,30 September 2026,99.925
2,Abrdn Asia Focus Plc GBX | GB00BFZ0WT29 | BFZ0WT2,2.250,31 May 2025,97.500
3,Allied Irish Banks plc GBP | XS0435957682 | BFMTY18,0.000,25 June 2035,61.000
4,Alpha Plus Holdings plc GBP | XS1379593566 | BZ5ZT30,5.000,31 March 2024,95.400
5,Anglian Water GBP | XS0089553282 | 0377986,6.875,21 August 2023,101.050
6,Anglian Water Services Financing plc GBP | XS0093312550 | 0482976,6.625,15 January 2029,108.675
7,Aviva plc GBP | XS1488459485 | BDF5PP0,4.375,12 September 2049,85.350
8,Aviva plc GBP | XS0138717441 | 3106518,6.125,14 November 2036,100.325
9,B.A.T. International Finance GBP | XS0468426266 | B5KP6X4,6.000,24 November 2034,91.425


In [354]:
df_issuer = df
print(df_issuer['Issuer'])
# Split "issuer" column by "|" character
df_issuer[['Name', 'ISIN', 'SEDOL']] = df_issuer['Issuer'].str.split('|', expand=True)

# Remove leading/trailing whitespace from the newly created columns
df_issuer['Name'] = df_issuer['Name'].str.strip()
df_issuer['ISIN'] = df_issuer['ISIN'].str.strip()
df_issuer['SEDOL'] = df_issuer['SEDOL'].str.strip()

# Drop the original "issuer" column
df_issuer.drop('Issuer', axis=1, inplace=True)

# Get a list of all column names except "Name", "ISIN", and "SEDOL"
other_columns = [col for col in df_issuer.columns if col not in ['Name', 'ISIN', 'SEDOL']]

# Reorder columns with "Name", "ISIN", and "SEDOL" as the first columns, followed by the remaining columns
df_issuer = df_issuer[['Name']+other_columns+['ISIN','SEDOL']]

# Display the updated DataFrame with additional columns
print(df_issuer)


0                                             3i Group GBP | XS0104440986 | 0924597
1                                      A2D Funding plc GBP | XS1103286305 | BQ8NZW9
2                                 Abrdn Asia Focus Plc GBX | GB00BFZ0WT29 | BFZ0WT2
3                               Allied Irish Banks plc GBP | XS0435957682 | BFMTY18
4                              Alpha Plus Holdings plc GBP | XS1379593566 | BZ5ZT30
5                                        Anglian Water GBP | XS0089553282 | 0377986
6                 Anglian Water Services Financing plc GBP | XS0093312550 | 0482976
7                                            Aviva plc GBP | XS1488459485 | BDF5PP0
8                                            Aviva plc GBP | XS0138717441 | 3106518
9                         B.A.T. International Finance GBP | XS0468426266 | B5KP6X4
10                    B.A.T. International Finance plc GBP | XS0522408599 | B3RS4H2
11                     Babcock International Group plc GBP | XS1499603170 | 

In [355]:
bond_underlyings=df_issuer['SEDOL'].values
bond_underlyings

array(['0924597', 'BQ8NZW9', 'BFZ0WT2', 'BFMTY18', 'BZ5ZT30', '0377986',
       '0482976', 'BDF5PP0', '3106518', 'B5KP6X4', 'B3RS4H2', 'BYX4LY7',
       '0051031', '0087177', '3079784', '0087025', '3149397', 'BYM5X34',
       'B0SRM51', 'B19ZMT5', 'B19ZPK7', '0379722', 'B1YYD83', 'BL6THL7',
       '0678087', 'BFWXVY2', 'BDR7W97', 'BYXJLY2', 'B04X7D6', 'BFXW063',
       'B3XR428', 'BFXWHQ2', 'B1YWFR0', '0301208', '3432451', '3304323',
       'B3F8XJ9', 'BMFTNK6', 'B7M3NB7', 'BR4R5Y4', 'BTL1KW6', '0347202',
       'B151HK3', 'B2NMK95', 'B0KCMD2', '3115912', 'B1VTTZ0', '0484110',
       '0039491', '0524287', '3414716', '0262468', '3227077', 'B4Y0FV9',
       'B3CPYK1', 'BGDQ0W8', 'B03HC45', '3342561', '0292669', '3235382',
       'B57N6P3', 'B3D0FP7', 'BJYHVQ2', 'BP95743', '0510491', '0365237',
       'BD20Y17', 'BL9ZVL0', 'B434V14', '0190536', 'B3MGZV2', 'B3Q4VK3',
       '3167913', 'BYYTRZ4', '0877505', 'B0712W1', 'B92X2L4', 'BYNXNT3',
       'BG88V04', '0491938', '3068157', '0134653', 

In [356]:
%%capture
%run gilts_yield_calculator.ipynb

In [357]:
def get_coupon_frequancy(frequency):
    switch_dict = {
        "Annually": 1,
        "Bi-annually": 2,
        "Quarterly": 4
    }
    return switch_dict.get(frequency, None)


In [358]:
# Define a function to generate link HTML
def generate_link(url):
    return f'<a href={url}>{url}</a>'



In [359]:
b_df = pd.DataFrame(columns = ['SEDOL', 'CouponFreq'])



In [360]:
def add_bond_to_data_frame(bond_underlying: str, concate_to_df):
    print('trying to scrap bond data {} from url '.format(bond_underlying))
    url_hl_search = f'https://www.hl.co.uk/shares/shares-search-results/{bond_underlying}'
    df = pd.read_html(url_hl_search)[0]
    df = df.T
    df.reset_index(drop=True, inplace=True)
    df.columns = df.iloc[0].str.rstrip(':')
    df.drop(0, inplace=True)
    df['Coupon_Freq'] = df['Coupon frequency'].map(get_coupon_frequancy)
    df['SEDOL'] = bond_underlying
    df['URL'] = generate_link(url_hl_search)
    df = df[['Coupon_Freq', 'SEDOL','URL']]
    concate_to_df = pd.concat([concate_to_df, df], ignore_index=False)
    return concate_to_df

In [361]:
bond_underlying = 'B03HC45'
#a = [add_bond_to_data_frame(b,b_df) for b in bond_underlyings]
#b_df=add_bond_to_data_frame(bond_underlying,b_df)
#b_df=b_df.style.format({'URL': generate_link})
#b_df



In [362]:
import asyncio
import aiohttp
import pandas as pd
import nest_asyncio

nest_asyncio.apply()  # Run this line if using Jupyter Notebook
async def fetch_data(bond_underlying: str):
    async with aiohttp.ClientSession() as session:
        url_hl_search = f'https://www.hl.co.uk/shares/shares-search-results/{bond_underlying}'
        async with session.get(url_hl_search) as response:
            html = await response.text()
            df= pd.read_html(html,flavor='bs4')[0]
            df = df.transpose()
            df.reset_index(drop=True, inplace=True)
            df.columns = df.iloc[0].str.rstrip(':')
            df.drop(0, inplace=True)
            df['CouponFreq'] = df['Coupon frequency'].map(get_coupon_frequancy)
            df['SEDOL'] = bond_underlying
            df = df[['CouponFreq', 'SEDOL']]
            return df

async def get_bond_coupon_freq(bond_underlying: str):
    try:
        print('trying to scrape bond data {} from url '.format(bond_underlying))
        df = await fetch_data(bond_underlying)
        return df
    except Exception as e:        
        print("error when calling bond {} m exception \t {}".format(bond_underlying,e))

async def main():
    global b_df
    tasks = [get_bond_coupon_freq(bond_underlying) for bond_underlying in bond_underlyings]
    results = await asyncio.gather(*tasks)
    b_df= pd.concat([b_df]+results)

await main()


trying to scrape bond data 0924597 from url 
trying to scrape bond data BQ8NZW9 from url 
trying to scrape bond data BFZ0WT2 from url 
trying to scrape bond data BFMTY18 from url 
trying to scrape bond data BZ5ZT30 from url 
trying to scrape bond data 0377986 from url 
trying to scrape bond data 0482976 from url 
trying to scrape bond data BDF5PP0 from url 
trying to scrape bond data 3106518 from url 
trying to scrape bond data B5KP6X4 from url 
trying to scrape bond data B3RS4H2 from url 
trying to scrape bond data BYX4LY7 from url 
trying to scrape bond data 0051031 from url 
trying to scrape bond data 0087177 from url 
trying to scrape bond data 3079784 from url 
trying to scrape bond data 0087025 from url 
trying to scrape bond data 3149397 from url 
trying to scrape bond data BYM5X34 from url 
trying to scrape bond data B0SRM51 from url 
trying to scrape bond data B19ZMT5 from url 
trying to scrape bond data B19ZPK7 from url 
trying to scrape bond data 0379722 from url 
trying to 

In [363]:
b_df

,SEDOL,CouponFreq
1,0924597,2
1,BQ8NZW9,2
1,BFZ0WT2,2
1,BFMTY18,1
1,BZ5ZT30,2
1,0377986,1
1,0482976,1
1,BDF5PP0,2
1,3106518,1
1,B5KP6X4,1


In [371]:
df_corp_bonds = pd.merge(df_issuer,b_df, on='SEDOL', how='outer')


In [372]:
df_corp_bonds=df_corp_bonds.rename(columns={'Coupon (%)': 'Coupon'})
df_corp_bonds=df_corp_bonds.dropna()
df_corp_bonds

,Name,Coupon,Maturity,Price,ISIN,SEDOL,CouponFreq
0,3i Group GBP,5.750,3 December 2032,102.290,XS0104440986,0924597,2
1,A2D Funding plc GBP,4.500,30 September 2026,99.925,XS1103286305,BQ8NZW9,2
2,Abrdn Asia Focus Plc GBX,2.250,31 May 2025,97.500,GB00BFZ0WT29,BFZ0WT2,2
3,Allied Irish Banks plc GBP,0.000,25 June 2035,61.000,XS0435957682,BFMTY18,1
4,Alpha Plus Holdings plc GBP,5.000,31 March 2024,95.400,XS1379593566,BZ5ZT30,2
5,Anglian Water GBP,6.875,21 August 2023,101.050,XS0089553282,0377986,1
6,Anglian Water Services Financing plc GBP,6.625,15 January 2029,108.675,XS0093312550,0482976,1
7,Aviva plc GBP,4.375,12 September 2049,85.350,XS1488459485,BDF5PP0,2
8,Aviva plc GBP,6.125,14 November 2036,100.325,XS0138717441,3106518,1
9,B.A.T. International Finance GBP,6.000,24 November 2034,91.425,XS0468426266,B5KP6X4,1


In [376]:
df_corp_bonds["Maturity"] = pd.to_datetime(df_corp_bonds["Maturity"], format="%d %B %Y")
df_corp_bonds['Ttm'] = df_corp_bonds["Maturity"].apply(lambda maturity: calculate_time_to_maturity(maturity))
df_corp_bonds

,Name,Coupon,Maturity,Price,ISIN,...,Ttm,AccruedInterest,DirtyPrice,DirtyYield,CleanYield
0,3i Group GBP,5.750,2032-12-03,102.290,XS0104440986,...,9.641096,1.674451,103.964451,5.221415,5.441601
1,A2D Funding plc GBP,4.500,2026-09-30,99.925,XS1103286305,...,3.460274,1.310440,101.235440,4.113301,4.523659
2,Abrdn Asia Focus Plc GBX,2.250,2025-05-31,97.500,GB00BFZ0WT29,...,2.126027,0.655220,98.155220,3.153951,3.480137
3,Allied Irish Banks plc GBP,0.000,2035-06-25,61.000,XS0435957682,...,12.200000,0.000000,61.000000,4.134807,4.134807
4,Alpha Plus Holdings plc GBP,5.000,2024-03-31,95.400,XS1379593566,...,0.958904,1.456044,96.856044,8.482867,10.155194
5,Anglian Water GBP,6.875,2023-08-21,101.050,XS0089553282,...,0.347945,1.996575,103.046575,-1.775901,3.780699
6,Anglian Water Services Financing plc GBP,6.625,2029-01-15,108.675,XS0093312550,...,5.756164,1.923973,110.598973,4.494385,4.861138
7,Aviva plc GBP,4.375,2049-09-12,85.350,XS1488459485,...,26.427397,1.274038,86.624038,5.323766,5.424911
8,Aviva plc GBP,6.125,2036-11-14,100.325,XS0138717441,...,13.591781,1.778767,102.103767,5.895715,6.089162
9,B.A.T. International Finance GBP,6.000,2034-11-24,91.425,XS0468426266,...,11.616438,1.742466,93.167466,6.872883,7.109019


In [ ]:
df_corp_bonds['AccruedInterest']=df_corp_bonds.apply(lambda x: calculate_accrued_interest(x['Coupon'],FACE_VALUE,x['Ttm'],x['CouponFreq']),axis=1)
df_corp_bonds['DirtyPrice']=df_corp_bonds.apply(lambda x: calculate_dirty_price(x['Price'],x['AccruedInterest']),axis=1)
df_corp_bonds['DirtyYield'] = df_corp_bonds.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['DirtyPrice'],x['CouponFreq']), axis=1)
df_corp_bonds['CleanYield'] = df_corp_bonds.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['Price'],x['CouponFreq']), axis=1)

In [378]:

df_less_than_Ttm = df_corp_bonds[df_corp_bonds['Ttm'] < 5]
df_less_than_Ttm=df_less_than_Ttm.sort_values("DirtyYield", ascending=False)
cols = ['Name','Coupon', 'Maturity', 'Price', 'Ttm','DirtyPrice', 'DirtyYield', 'CleanYield']
df_less_than_Ttm = df_less_than_Ttm[cols]
df_less_than_Ttm

,Name,Coupon,Maturity,Price,Ttm,DirtyPrice,DirtyYield,CleanYield
39,Eros Media World Plc GBP,8.500,2026-04-15,41.875,3.000000,44.350275,43.384772,46.169276
63,International Personal Finance Plc GBP,12.000,2027-12-12,100.375,4.660274,103.869505,10.918599,11.892873
37,EnQuest plc GBP,9.000,2027-10-27,93.575,4.534247,96.195879,10.065528,10.830584
30,Co-operative Group Ltd GBP,7.500,2026-07-08,94.100,3.230137,96.278082,8.875375,9.714768
4,Alpha Plus Holdings plc GBP,5.000,2024-03-31,95.400,0.958904,96.856044,8.482867,10.155194
50,Hammerson GBP,6.000,2026-02-23,93.000,2.860274,94.742466,8.133681,8.877718
40,Esure Group Plc GBP,6.750,2024-12-19,96.000,1.679452,97.965659,8.069485,9.379554
32,Daily Mail & General Trust GBP,6.375,2027-06-21,93.125,4.183562,94.976370,7.829192,8.390971
93,PGH Capital Ltd GBP,6.625,2025-12-18,95.950,2.676712,97.873973,7.531469,8.376271
29,Co-operative Group Ltd GBP,11.000,2025-12-22,105.400,2.687671,108.594521,7.359646,8.662427
